Spark TensorFlow Distributor is a python library which implements the barrier execution mode of spark to implement distributed TensorFlow training on top of the spark 3.0 cluster. You need to install this library as part of the pyspark environment.

https://github.com/tensorflow/ecosystem/tree/master/spark/spark-tensorflow-distributor.

Note:
If you are using Mac, see, following error might show after code crashing:
[__NSPlaceholderDate initialize] may have been in progress in another thread when fork() was called.

Solution:
Add following line in .zshrc or .bash_profile
OBJC_DISABLE_INITIALIZE_FORK_SAFETY=YES
https://stackoverflow.com/questions/50168647/multiprocessing-causes-python-to-crash-and-gives-an-error-may-have-been-in-progr

2.  Use "use_gpu=False" if your machine does not have Nvdia GPU or coda is not enabled. 
MirroredStrategyRunner(num_slots=8, use_gpu=False).run(train)


In [1]:
#!pip install tensorflow

In [2]:
#!pip install spark-tensorflow-distributor

In [3]:
from spark_tensorflow_distributor import MirroredStrategyRunner

# Adapted from https://www.tensorflow.org/tutorials/distribute/multi_worker_with_keras
def train():
    import tensorflow as tf
    import uuid

    BUFFER_SIZE = 10000
    BATCH_SIZE = 64

    def make_datasets():
        (mnist_images, mnist_labels), _ = \
            tf.keras.datasets.mnist.load_data(path=str(uuid.uuid4())+'mnist.npz')

        dataset = tf.data.Dataset.from_tensor_slices((
            tf.cast(mnist_images[..., tf.newaxis] / 255.0, tf.float32),
            tf.cast(mnist_labels, tf.int64))
        )
        dataset = dataset.repeat().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
        return dataset

    def build_and_compile_cnn_model():
        model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(10, activation='softmax'),
        ])
        model.compile(
            loss=tf.keras.losses.sparse_categorical_crossentropy,
            optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
            metrics=['accuracy'],
        )
        return model

    train_datasets = make_datasets()
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
    train_datasets = train_datasets.with_options(options)
    multi_worker_model = build_and_compile_cnn_model()
    multi_worker_model.fit(x=train_datasets, epochs=3, steps_per_epoch=5)

MirroredStrategyRunner(num_slots=8, use_gpu=False).run(train)

Doing CPU training...
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/28 10:54:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Will run with 8 Spark tasks.
Distributed training in progress...
View Spark executor stderr logs to inspect training...
2023-05-28 10:54:45.258380: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 10:54:45.259394: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 10:54:45.259434: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, 

2023-05-28 10:54:57.251132: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [60000]
	 [[{{node Placeholder/_1}}]]
2023-05-28 10:54:57.252858: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [60000]
	 [[{{node Placeholder/_1}}]]
2023-05-28 10:54:57.262522: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [60000

5/5 [==============================] - 2s 88ms/step - loss: 2.2921 - accuracy: 0.1250
Epoch 2/3
Epoch 2/3
Epoch 2/3
Epoch 2/3
Epoch 2/3
Epoch 2/3
Epoch 2/3
Epoch 2/3
5/5 [==============================] - 0s 58ms/step - loss: 2.3044 - accuracy: 0.0875
Epoch 3/3
Epoch 3/3
Epoch 3/3
Epoch 3/3
Epoch 3/3
Epoch 3/3
Epoch 3/3
Epoch 3/3
5/5 [==============================] - 0s 59ms/step - loss: 2.3097 - accuracy: 0.0844
Training with 8 slots is complete!                                              
